## 영화 추천을 위한 영화 데이터 및 리뷰정보 크롤링

1. 왓차피디아 
- https://pedia.watcha.com/ko-KR

2. 영화개봉정보 - 영화관입장권통합전산망
- https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do#none


In [1]:
# !pip freeze > requirements.txt
#!pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests


from urllib.parse import quote

import time
#time.sleep을 위한 난수생성
import random

from tqdm import tqdm

# text 전처리
import re

# 마우스 커서 이동을 위한 패키지
import pyautogui

In [3]:
movie = pd.read_excel('영화정보 리스트_2024-03-19.xls', header=4)
movie.head(5)

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,남은 인생 10년,The Last 10 Years,2022.0,일본,장편,멜로/로맨스,개봉,후지이 미치히토,NaN
1,달짝지근해: 7510,HONEYSWEET,2022.0,한국,장편,코미디,개봉,이한,(주)무비락
2,폭로,Havana,2022.0,한국,장편,"범죄,스릴러",개봉,홍용호,(주)로그라인스튜디오
3,어디로 가고 싶으신가요,Where Would You Like to Go?,2023.0,한국,장편,드라마,개봉,김희정,(주)인벤트스톤
4,만해 한용운 님의침묵,Silence of Thy,2022.0,한국,장편,드라마,개봉,한명구,주식회사 시네마서울


In [4]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영화명      1547 non-null   object 
 1   영화명(영문)  794 non-null    object 
 2   제작연도     1516 non-null   float64
 3   제작국가     1547 non-null   object 
 4   유형       1547 non-null   object 
 5   장르       1493 non-null   object 
 6   제작상태     1546 non-null   object 
 7   감독       1242 non-null   object 
 8   제작사      632 non-null    object 
dtypes: float64(1), object(8)
memory usage: 108.9+ KB


In [5]:
movie['장르'].fillna('', inplace=True)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_9356\3068823807.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movie['장르'].fillna('', inplace=True)


In [6]:
movie = movie[~movie['장르'].str.contains('성인')]

In [7]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 775 entries, 0 to 1546
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영화명      775 non-null    object 
 1   영화명(영문)  658 non-null    object 
 2   제작연도     753 non-null    float64
 3   제작국가     775 non-null    object 
 4   유형       775 non-null    object 
 5   장르       775 non-null    object 
 6   제작상태     775 non-null    object 
 7   감독       624 non-null    object 
 8   제작사      257 non-null    object 
dtypes: float64(1), object(8)
memory usage: 60.5+ KB


In [8]:
movie['제작연도'].unique()

array([2022., 2023., 2020., 2021., 2004.,   nan, 2019., 1995., 1990.,
       1988., 1964., 1980., 2018., 2015., 1948., 1999., 1996.])

In [9]:
movie['제작연도'] =  movie['제작연도'].fillna(0).apply(lambda x: str(int(np.round(x))))

In [10]:
movie['장르'].unique()

array(['멜로/로맨스', '코미디', '범죄,스릴러', '드라마', '코미디,드라마,멜로/로맨스', '다큐멘터리',
       '사극,드라마', '멜로/로맨스,판타지,드라마', '판타지,드라마', '드라마,멜로/로맨스,범죄',
       '멜로/로맨스,코미디', '드라마,기타', '사극,액션', '드라마,멜로/로맨스,뮤지컬,공연', '애니메이션',
       '드라마,멜로/로맨스', '드라마,사극,스릴러,전쟁', '코미디,드라마', '공포(호러),코미디',
       '미스터리,드라마,스릴러', '드라마,미스터리', '드라마,액션,범죄', 'SF,드라마,스릴러',
       '드라마,판타지,멜로/로맨스', '공연,다큐멘터리', '', '미스터리,범죄,드라마', '스릴러',
       '멜로/로맨스,드라마', '공연', '공포(호러)', '멜로/로맨스,드라마,판타지', '드라마,범죄', '범죄',
       '판타지,어드벤처,애니메이션', '전쟁,액션', '스릴러,멜로/로맨스', '공포(호러),미스터리',
       '액션,어드벤처,판타지', '미스터리,스릴러,드라마', '액션', 'SF,액션', '범죄,액션',
       '공포(호러),스릴러', '코미디,액션', '미스터리,스릴러', '어드벤처,드라마,코미디',
       '애니메이션,어드벤처,판타지', '스릴러,드라마,코미디', '전쟁,드라마', '드라마,멜로/로맨스,가족',
       '액션,드라마,스릴러', '액션,드라마', '애니메이션,어드벤처', '범죄,드라마,스릴러',
       '애니메이션,판타지,어드벤처', '액션,스릴러,전쟁', '미스터리,공포(호러)', 'SF,스릴러',
       '다큐멘터리,드라마,가족', '판타지,액션', '다큐멘터리,범죄', '범죄,액션,스릴러', '스릴러,공포(호러)',
       'SF,액션,드라마', '서부극(웨스턴),코미디,액션', '액션,어드벤처,SF,스릴러', '액션,스릴러',
       '액션,범죄', '드라마,코미디', '

In [11]:
movie_list = movie['영화명'].to_list()
movie_list

['남은 인생 10년',
 '달짝지근해: 7510',
 '폭로',
 '어디로 가고 싶으신가요',
 '만해 한용운 님의침묵',
 '나비효과',
 '울지마 엄마',
 '연악: 나의 운명',
 '어느 날 그녀가 우주에서',
 '3일의 휴가',
 '흠뻑채운 젖은 속살',
 '탐욕에 젖은 섹스',
 '탐욕스런 동침',
 '타락한 유부녀의 유혹',
 '유부녀의 욕망 속살',
 '유부녀의 비밀스런 사정',
 '옆집 유부녀의 속사정',
 '아버님이 채운 유부녀',
 '자우림, 더 원더랜드',
 '위험사회',
 '서울의 봄',
 '싱글 인 서울',
 '콘크리트 유토피아',
 '노량: 죽음의 바다',
 '사랑의 불시착: 라이브 인 서울',
 '두다다쿵: 후후섬의 비밀',
 '나의 피투성이 연인',
 '듣보인간의 생존신고',
 '어른 김장하',
 '돼지의 최후',
 '겨울 이야기',
 '드림',
 '손',
 '비밀',
 '신세계로부터',
 '솔라 플라워',
 '르네에게',
 '익스트림 페스티벌',
 '다음 소희',
 '괴인',
 '돈 룩 디퍼',
 '사랑엔 정의가 없다',
 '그대들은 어떻게 살 것인가',
 '청춘 돼지는 외출하는 여동생의 꿈을 꾸지 않는다',
 '너와 나',
 '말하고 싶은 비밀',
 '상견니',
 '류이치 사카모토: 오퍼스',
 '절해고도',
 '미끼',
 '도티와 영원의 탑',
 '사위 사랑은 장모 4',
 '여배우들 : 바꿔할래?',
 '그녀의 취미생활',
 '라방',
 '오늘 출가합니다',
 '리향',
 '자매의 스와핑',
 '참을 수 없는 탐욕',
 '바지 내린 욕정녀',
 '몸으로 받은 상사',
 '우리 돈 벌레',
 '옆집 아저씨와 같은 침대에 누워 있다',
 '새엄마 잠깐 거긴 안돼',
 '멋대로 들어온 돌싱녀 : 하고 싶어요',
 '남편 말고 의붓아들 : 속궁합이 잘 맞아',
 '쉽게 넣는 그녀 무삭제판',
 '가랑이 채운 처형 무삭제판',
 '르네상스 필름 바이 비욘세',
 '아이엠 장모에요',
 '며느리의 반복 된 숨소리 : 나를 흥분하게 해

In [12]:
year_list = movie['제작연도'].to_list()
year_list

['2022',
 '2022',
 '2022',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2020',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2023',
 '2021',
 '2022',
 '2023',
 '2022',
 '2022',
 '2021',
 '2023',
 '2023',
 '2004',
 '2023',
 '2021',
 '2023',
 '2021',
 '2022',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2023',
 '2021',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2021',
 '2023',
 '2023',
 '2023',
 '2023',
 '0',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '0',
 '2023',
 '2023',
 '2022',
 '2022',
 '2023',
 '2023',
 '2022',
 '2022',
 '2020',
 '2022',
 '2021',
 '2022',
 '2022',
 '2020',
 '2023',
 '2023',
 '2023',
 '2022',
 '2023',
 '2022',
 '2022',
 '2022',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2023',
 '2023',
 '2019',
 '2023',
 '2023',
 '2022',
 '2023',
 '2022',
 '2023',
 '2022',
 '2021',
 '2023',
 '2022',
 '2023'

In [13]:
movie.reset_index(drop = True, inplace=True)

In [14]:
movie

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,남은 인생 10년,The Last 10 Years,2022,일본,장편,멜로/로맨스,개봉,후지이 미치히토,NaN
1,달짝지근해: 7510,HONEYSWEET,2022,한국,장편,코미디,개봉,이한,(주)무비락
2,폭로,Havana,2022,한국,장편,"범죄,스릴러",개봉,홍용호,(주)로그라인스튜디오
3,어디로 가고 싶으신가요,Where Would You Like to Go?,2023,한국,장편,드라마,개봉,김희정,(주)인벤트스톤
4,만해 한용운 님의침묵,Silence of Thy,2022,한국,장편,드라마,개봉,한명구,주식회사 시네마서울
...,...,...,...,...,...,...,...,...,...
770,패닉 런,The Desperate Hour,2021,미국,장편,스릴러,개봉,필립 노이스,NaN
771,처음 꽃향기를 만난 순간,Fragrance of the First Flower,2021,대만,장편,"드라마,멜로/로맨스",개봉,덩이한,NaN
772,뮤지컬 번지점프를 하다,NaN,2022,한국,장편,"뮤지컬,공연",개봉,윤영호,씨제이 씨지브이(CJ CGV)(주)
773,뮤지컬 배니싱,NaN,2020,한국,장편,"뮤지컬,공연",개봉,이헌재,씨제이 씨지브이(CJ CGV)(주)


In [15]:
### 데이터 프레임화

base_url = 'https://pedia.watcha.com/ko-KR/searches/movies?query='

movie_info = []
movie_review = []

for i in range(300):
#for i in range(200, 220):
    movie_title = movie['영화명'][i]
    movie_year = movie['제작연도'][i]

    url = base_url + movie_title

    driver = webdriver.Chrome()

    driver.get(url)
    #driver.set_window_position(-1800,10)

    # 화면이 뜰때까지 기다리기
    driver.implicitly_wait(10)
    # 창 최대화
    # driver.maximize_window()
    # time.sleep(2.4)
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    #################################################################################################################
    ### 영화가 검색이 안되는 경우 - try except로 
    try: 
        #################################################################################################################
        ### 같은 제목의 영화에서 제작연도가 같은 영화를 선택
        same_lis = driver.find_element(By.CLASS_NAME, 'e1493pgd7').find_elements(By.CLASS_NAME, 'e1ic68ft1')

        cnt = 0
        for li in same_lis:
            li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0]
            if li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0] != movie_year:
                cnt += 1
            else :
                # print(li.find_element(By.CLASS_NAME, 'e1gexyxq4').text.strip())
                # print(cnt)
                break

        # 제작연도가 같은 영화가 없을 경우에 첫번째 영화를 선택
        try:
            same_lis[cnt].click()
        except:
            same_lis[0].click()

        #################################################################################################################
        ### 영화 정보 크롤링
        # 영화 소개
        summary = driver.find_element(By.CLASS_NAME, 'e1yew2864').text.strip().replace('\n', '')

        # 감독/출연진
        actor_list = []
        person_lis = driver.find_element(By.CLASS_NAME, 'e1n40jsk1').find_elements(By.CLASS_NAME, 'e4f2vpe2')
        for person in person_lis:
            actor_list.append(person.find_element(By.TAG_NAME, 'a').get_attribute('title'))

        # 전체 평점
        total_rating = driver.find_element(By.CLASS_NAME, 'e15eo6m110').text

        # 전체 평점 리뷰어수
        rating_cnt = driver.find_element(By.CLASS_NAME, 'e15eo6m19').text.split('\n')[-1]
        

        # 정보 1
        info1 = driver.find_element(By.CLASS_NAME, 'e1yew28613').find_elements(By.CLASS_NAME, 'e1yew28611')[1].text

        # 정보 2
        info2 = driver.find_element(By.CLASS_NAME, 'e1yew28613').find_elements(By.CLASS_NAME, 'e1yew28611')[2].text

        time.sleep(round(random.uniform(1.3, 2.2), 3))
        movie_info.append({'title' : movie_title,
                        'movie_num' : i,
                        'summary' : summary,
                        'actor_list': actor_list,
                        'rating' : total_rating,
                        'rating_cnt' : rating_cnt,
                        'info1' : info1,
                        'info2' : info2})
        
        #################################################################################################################
        # 리뷰 페이지 이동
        driver.find_element(By.CLASS_NAME, 'ehapiph0').click()
        time.sleep(round(random.uniform(0.7, 1.2), 3))

        # 스크롤 다운    
        for _ in range(20):
            last_height = driver.execute_script("return document.body.scrollHeight")

        # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
            driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
            time.sleep(round(random.uniform(1.0, 1.5), 3))
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
            time.sleep(round(random.uniform(0.3, 1.2), 3))

        # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break

        time.sleep(round(random.uniform(1.3, 2.2), 3))


        num = 0
        review_list = driver.find_elements(By.CLASS_NAME, 'egj9y8a4')
        for review_li in review_list:
            
            # 평점이 없는 경우 발생 - ex)보고싶어요
            try:
                # 평점
                review_rating = review_li.find_element(By.CLASS_NAME, 'egj9y8a0').text.strip()

                #사용자명
                reviewer = review_li.find_element(By.CLASS_NAME, 'e10cf2lr1').text.strip()

                # 리뷰
                review = review_li.find_element(By.CLASS_NAME, 'e1hvy88212').text.strip().replace('\n', '')

                # num
                num += 1

                movie_review.append({'title' : movie_title,
                                    'movie_num' : i,
                                    'review_num' : num,
                                    'reviewer' : reviewer,
                                    'review_rating' : review_rating,
                                    'review' : review})
            except:
                continue
        time.sleep(round(random.uniform(1.3, 2.2), 3))

        driver.close()
    except:
        continue

KeyboardInterrupt: 

In [ ]:
df_info = pd.DataFrame(movie_info)
df_review = pd.DataFrame(movie_review)

In [ ]:
df_info.to_csv('movie_info.csv', index = False)
df_review.to_csv('movie_review.csv', index = False)

In [ ]:
df_info

,title,movie_num,summary,actor_list,rating,rating_cnt,info1,info2
0,컨트리 카바레,201,"압류 위기에 처해 위태로운 ‘데이비드’의 농장, 우연히 들른 카바레에서 본 스타 댄...","[장 피에르 아메리스(감독), 사브리나 우아자니(주연), 알반 이바노프(주연), 가...",1.6,(4명),2022 · 코미디/드라마 · 프랑스,1시간 49분 · 12세
1,1947 보스톤,202,“나라가 독립을 했으면 당연히 우리 기록도 독립이 되어야지!” 1936년 베를린 올...,"[강제규(감독), 하정우(주연 | 기정), 임시완(주연 | 윤복), 배성우(주연 |...",2.9,"(6,470명)",2023 · 드라마/스포츠/액션/시대극 · 한국,1시간 48분 · 12세
2,데블스 맨: 제1특전단,203,2차 세계대전에서 평범한 청년이 제1특전단에 들어가서 지옥훈련을 받고 전쟁터로 가서...,"[데이브 브레스나한(감독), 케이시 엘리엇(주연), 케일럽 하펜(주연)]",3.0,(1명),2023 · 전쟁/액션 · 미국,1시간 31분 · 12세
3,버텨내고 존재하기,204,음악을 만드는 사람의 얼굴 같은 음악공간을 지켜내는 사람의 얼굴 같은 공간1935년...,"[권철(감독), 김일두(출연), 김사월(출연), 곽푸른하늘(출연), 불나방스타쏘세지...",3.5,(343명),2022 · 다큐멘터리 · 한국,1시간 4분 · 전체
4,이빨요정 비올레타: 요정나라로 돌아갈래!,205,전 세계 아이들이 기다리는 이빨요정이 온다! 아이들이 자신의 빠진 유치를 베개 밑에...,"[캐롤라인 오리거(감독), 옐라 하제(성우), 리자-마리 코롤(성우), 메리 머레이...",2.6,(30명),"2022 · 애니메이션/판타지/모험/코미디/가족 · 룩셈부르크, 독일",1시간 25분 · 전체
5,아워 프렌드,206,두 딸과 행복한 일상을 보내던 니콜(다코타 존슨)과 맷(케이시 애플렉) 부부. 어느...,"[가브리엘라 카우퍼스웨이트(감독), 다코타 존슨(주연 | 니콜), 케이시 애플렉(주...",3.1,(88명),2019 · 로맨스/드라마 · 미국,2시간 5분 · 15세
6,마크로스 플러스 -무비 에디션-,207,2040년 인류 최초의 이민 행성 에덴. 7년 전 소꿉친구 ‘뮹’을 두고 서로를 등...,"[카와모리 쇼지(감독), 와타나베 신이치로(감독), 야마자키 타쿠미(주연), 이시즈...",3.7,"(3,112명)","1994 · 애니메이션 · 미국, 일본",1시간 54분 · 15세
7,극장판 파워 디지몬 더 비기닝,208,모험의 날로부터 10년이 지난 2012년. 선택받은 아이들과 파트너 디지몬은 변함없...,"[타구치 토모히사(감독), 카타야마 후쿠쥬로(성우), 노다 준코(성우), 란즈베리 ...",2.6,(475명),2023 · 애니메이션/모험/가족 · 일본,1시간 20분 · 12세
8,녹야,209,인천 여객항 보안검색대에서 근무하는 이방인 ‘진샤’(판빙빙). 낯선 타지에서 쳇바퀴...,"[한슈아이(감독), 판빙빙(주연), 이주영(주연), 김영호(조연)]",2.4,"(1,308명)","2023 · 스릴러/미스터리/드라마 · 중국, 홍콩",1시간 32분 · 15세
9,괴물,210,“우리 동네에는 괴물이 산다” 싱글맘 사오리(안도 사쿠라)는 아들 미나토(쿠로카와 ...,"[고레에다 히로카즈(감독), 안도 사쿠라(주연), 나가야마 에이타(주연), 쿠로카와...",4.3,(5.8만명),2023 · 드라마/미스터리 · 일본,2시간 6분 · 12세


In [ ]:
df_review

,title,movie_num,review_num,reviewer,review_rating,review
0,컨트리 카바레,201,1,땡만,0.5,진심개에바
1,1947 보스톤,202,1,웬디,3.5,"보고왔는데,보지도 않고 엄복동거리는 댓글들 어딜가나 있네? 재밌지도 않은 드립 개나..."
2,1947 보스톤,202,2,이동진 평론가,2.0,정해진 지점들 사이로 그저 터덜터덜 달린다.
3,1947 보스톤,202,3,박준영 (영화 담다),3.0,안다면 아는 내용이고 모를수도 있지만 전개 하나만큼은 빨라서 좋았다. 실화라는 특성...
4,1947 보스톤,202,4,TERU,2.5,Good : 런복동은 너무 나갔다. 임시완은 빛났다.Caution : 국뽕도 나라가...
...,...,...,...,...,...,...
602,사채소년,219,94,남형진,1.5,교복을 입는 미성년이 어슬프게 성년의 흉내를 내면 안되는 법이다
603,사채소년,219,95,타노스,1.0,이게 뭔..
604,사채소년,219,96,이정곤,2.0,초반의 발칙함을 급속도로 잃어간다.
605,사채소년,219,97,l00K,3.0,그렇게까지 나쁘진 않았다뭐 나름 재밌는 상상이었고 뭘해도 세상은 바뀌지 않는다는 느...


In [ ]:
movie['영화명'][215]

'총각파티 ： 은밀한 서비스'